In [1]:
import pandas as pd
import numpy as np
from astropy.coordinates import SkyCoord
from astropy import units as u

import sys
sys.path.append('../')
import os
from src.classification import get_match_label_simple
from src.data import get_data_basic_matches
from sklearn.model_selection import train_test_split
from src.utils import transform_features, normalize_train_test

from joblib import load

In [2]:
df_all_model = pd.read_parquet('../scripts/nway_csc21_gaia3_full_neg_study_dis_niter200.parquet')

In [3]:
benchmark_ids = load('../scripts/jobs/models/neg_study_dis_niter200_withint_with_int_5X_lgbm_0-3_20241113_235113/benchmark_ids.joblib')

In [4]:
def get_train_val_test_splits(df_all_model, benchmark_ids, range_offaxis='0-3', separation=1.3):
   # get initial positives and split test set
   df_pos, _ = get_data_basic_matches(df_all_model, range_offaxis, separation)
   cscids = df_pos['csc21_name'].unique()
   cscids_train_val, cscids_test = train_test_split(cscids, test_size=0.2, random_state=42)
   
   # get train/val split from filtered train_val data
   df_train_val = df_all_model[df_all_model['csc21_name'].isin(cscids_train_val)]
   train_pos, _ = get_data_basic_matches(df_train_val, range_offaxis, separation)
   train_val_cscids = train_pos['csc21_name'].unique()
   cscids_train, cscids_val = train_test_split(train_val_cscids, test_size=0.2, random_state=42)
   
   assert set(benchmark_ids) == set(cscids_test)
   
   # get final datasets
   splits = {}
   for name, ids in [('train', cscids_train), ('val', cscids_val), ('test', cscids_test)]:
       data = df_all_model[df_all_model['csc21_name'].isin(ids)]
       pos, neg = get_data_basic_matches(data, range_offaxis, separation)
       splits[name] = {'pos': pos, 'neg': neg, 'full': data}
       
   return splits

splits = get_train_val_test_splits(df_all_model, benchmark_ids)

Range 0-3: 30279 positives, 310020 negatives
Range 0-3: 24223 positives, 245627 negatives
Range 0-3: 19378 positives, 195135 negatives
Range 0-3: 4845 positives, 50492 negatives
Range 0-3: 6056 positives, 64393 negatives


In [5]:
def validate_splits(splits, model_path):
   """validate train/val splits match saved model data"""
   
   # combine pos/neg sets
   val_data = splits['val']['full']
   train_data = splits['train']['full']
   
   # load saved validation data
   X_eval_saved = load(os.path.join(model_path, 'X_eval.joblib'))
   y_eval_saved = load(os.path.join(model_path, 'y_eval.joblib'))
   
   # prepare validation data
   val_data['eval_label'] = np.where(val_data['match_flag'] == 1, 1, 0)
   
   # preprocess
   X_train, _ = transform_features(train_data, log_transform=False, model_type='lgbm')
   X_val, cat_features = transform_features(val_data, log_transform=False, model_type='lgbm')
   _, X_val_norm, _ = normalize_train_test(X_train, X_val, method='none', 
                                         categorical_features=cat_features)
   
   # verify
   assert X_eval_saved.equals(X_val_norm)
   assert np.array_equal(y_eval_saved, val_data['eval_label'].values)
   
   return True

validate_splits(splits, '../scripts/jobs/models/neg_study_dis_niter200_withint_with_int_5X_lgbm_0-3_20241113_235113')

/tmp/ipykernel_1966374/694336655.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_data['eval_label'] = np.where(val_data['match_flag'] == 1, 1, 0)


True

In [6]:
# get the test data + everything from the dataset that was not in train and val
test_data = splits['test']['full']

# now get everything that was not in train and val
train_val_data = pd.concat([splits['train']['full'], splits['val']['full']])
train_val_ids = train_val_data['csc21_name'].unique()
not_train_val_data = df_all_model[~df_all_model['csc21_name'].isin(train_val_ids)].copy()

# check if test is IN not_train_val_data
assert set(test_data['csc21_name'].unique()) <= set(not_train_val_data['csc21_name'].unique())

In [7]:
# Define the center of the Orion Nebula region
orion_center = SkyCoord(ra=83.8210 * u.deg, dec=-5.3944 * u.deg, frame='icrs')

# Create SkyCoord objects for all sources
orion_source_coords = SkyCoord(ra=df_all_model['csc21_ra'].values * u.deg, dec=df_all_model['csc21_dec'].values * u.deg, frame='icrs')

# Calculate separations
orion_separations = orion_source_coords.separation(orion_center).to(u.arcmin)

# Filter the dataframe
df_all_model['separation_from_orion'] = orion_separations
orion_sources_in_region = df_all_model[orion_separations <= 30 * u.arcmin].copy()
orion_cscid_list = orion_sources_in_region['csc21_name'].str.replace('_', ' ').str.strip().unique().tolist()
orion_sources_in_region['num_possible_counterparts'] = orion_sources_in_region.groupby('csc21_name')['gaia3_source_id'].transform('count')


In [8]:
df_in_crit = get_match_label_simple(orion_sources_in_region, p_threshold=0.35)

In [9]:
df_in_crit_test_and_more = df_in_crit[df_in_crit['csc21_name'].isin(not_train_val_data['csc21_name'].unique())]

In [10]:
df_in_crit_test_and_more.csc21_name.nunique()

1412

In [11]:
# check if all the cases with min_theta_mean < 3 and separation <1.3 are in the test set

assert set(df_in_crit_test_and_more.query('min_theta_mean < 3 and separation<1.3 and p_any>0.9')['csc21_name'].unique()) <= set(test_data['csc21_name'].unique())

In [12]:
def create_performance_metrics(df):
   """compute match statistics between nway and ml model"""
   metrics = {}
   
   # base counts
   metrics['N_CSC'] = df['csc21_name'].nunique()
   metrics['N_yNWAY'] = df[df['p_any'] > 0.5]['csc21_name'].nunique()
   
   # combined criteria using label column
   ok_matches = (df['p_any'] > 0.5) & (df['label'] == 1)
   metrics['N_OK'] = df[ok_matches]['csc21_name'].nunique()
   
   no_ml = (df['p_any'] > 0.5) & ~df.groupby('csc21_name')['label'].transform(any)
   metrics['N_NoML'] = df[no_ml]['csc21_name'].nunique()
   
   flip = (df['p_any'] <= 0.5) & (df['label'] == 1)
   metrics['N_FLIP'] = df[flip]['csc21_name'].nunique()
   
   none = (df['p_any'] <= 0.5) & ~df.groupby('csc21_name')['label'].transform(any)
   metrics['N_NONE'] = df[none]['csc21_name'].nunique()
   
   # single/multiple match counts 
   ok_single = ok_matches & (df.groupby('csc21_name')['label'].transform(sum) == 1)
   metrics['N_yNWAY+yMLeq1'] = df[ok_single]['csc21_name'].nunique()
   metrics['N_yNWAY+yMLgt1'] = metrics['N_OK'] - metrics['N_yNWAY+yMLeq1']
   
   flip_single = flip & (df.groupby('csc21_name')['label'].transform(sum) == 1)
   metrics['N_FLIPeq1'] = df[flip_single]['csc21_name'].nunique()
   metrics['N_FLIPgt1'] = metrics['N_FLIP'] - metrics['N_FLIPeq1']

   table = f"""
N_CSC = {metrics['N_CSC']}
N_yNWAY = {metrics['N_yNWAY']}
N_OK = {metrics['N_OK']}
N_NoML = {metrics['N_NoML']}
N_FLIP = {metrics['N_FLIP']}
N_NONE = {metrics['N_NONE']}
N_yNWAY+yMLeq1 = {metrics['N_yNWAY+yMLeq1']}
N_yNWAY+yMLgt1 = {metrics['N_yNWAY+yMLgt1']}
N_FLIPeq1 = {metrics['N_FLIPeq1']}
N_FLIPgt1 = {metrics['N_FLIPgt1']}
"""
   
   return metrics, table

In [13]:
stats_vk, email_vk = create_performance_metrics(df_in_crit_test_and_more)

print(email_vk)


N_CSC = 1412
N_yNWAY = 1006
N_OK = 952
N_NoML = 54
N_FLIP = 44
N_NONE = 362
N_yNWAY+yMLeq1 = 920
N_yNWAY+yMLgt1 = 32
N_FLIPeq1 = 36
N_FLIPgt1 = 8



/tmp/ipykernel_1966374/103789065.py:23: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ok_single = ok_matches & (df.groupby('csc21_name')['label'].transform(sum) == 1)
/tmp/ipykernel_1966374/103789065.py:27: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  flip_single = flip & (df.groupby('csc21_name')['label'].transform(sum) == 1)


In [20]:
def get_gaia_multiple_n(df):
   """count total gaia matches in multiple match cases"""
   # get sources with multiple matches
   ok_multiples = (df['p_any'] > 0.5) & (df.groupby('csc21_name')['label'].transform(sum) > 1)
   flip_multiples = (df['p_any'] <= 0.5) & (df.groupby('csc21_name')['label'].transform(sum) > 1)

   # count matches
   n_ok_gaia = df[ok_multiples & (df['label'] == 1)]['gaia3_source_id'].nunique()
   n_flip_gaia = df[flip_multiples & (df['label'] == 1)]['gaia3_source_id'].nunique()
   
   return {
       'N_yNWAY+yMLgt1_gaia': n_ok_gaia,
       'N_FLIPgt1_gaia': n_flip_gaia
   }

In [21]:
get_gaia_multiple_n(df_in_crit_test_and_more)

/tmp/ipykernel_1966374/3518964572.py:4: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ok_multiples = (df['p_any'] > 0.5) & (df.groupby('csc21_name')['label'].transform(sum) > 1)
/tmp/ipykernel_1966374/3518964572.py:5: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  flip_multiples = (df['p_any'] <= 0.5) & (df.groupby('csc21_name')['label'].transform(sum) > 1)


{'N_yNWAY+yMLgt1_gaia': 65, 'N_FLIPgt1_gaia': 16}

In [16]:
def analyze_match_cases(df, df_all_stacks):
   """export csvs for each match case type"""
   output_cols = [
       'csc21_name', 'csc21_ra', 'csc21_dec', 'min_theta_mean',
       'detect_stack_id', 'gaia3_source_id', 'p_i', 'p_any', 
       'p_match_ind', 'separation', 'match_flag', 'label',
       'phot_g_mean_mag', 'phot_bp_mean_mag', 'phot_rp_mean_mag', 'bp_rp'
   ]

   # initialize cases
   N_yNWAY = []
   N_OK = []
   N_NoML = [] 
   N_FLIP = []
   N_NONE = []
   
   for csc_id, group in df.groupby('csc21_name'):
       nway_confident = group['p_any'].max() > 0.5
       ml_matches = group[group['label'] == 1]
       
       if nway_confident:
           N_yNWAY.append(group)
           if len(ml_matches) > 0:
               N_OK.append(group)
           else:
               N_NoML.append(group)
       else:
           if len(ml_matches) > 0:
               N_FLIP.append(group)
           else:
               N_NONE.append(group)
   
   # save cases to csv
   cases = {
       'N_yNWAY': N_yNWAY,
       'N_OK': N_OK,
       'N_NoML': N_NoML,
       'N_FLIP': N_FLIP,
       'N_NONE': N_NONE
   }
   
   for case_name, case_data in cases.items():
       if case_data:
           df_case = (pd.concat(case_data)
                     .merge(df_all_stacks[['name', 'detect_stack_id']], 
                           left_on='csc21_name', right_on='name', 
                           how='left')
                     [output_cols])
           df_case.to_csv(f'outputs/coup_{case_name}_trs0.35.csv', index=False)
           
   return {k: len(set(pd.concat(v)['csc21_name'])) if v else 0 
           for k,v in cases.items()}

In [15]:
# read vot to pandas ../data/all_stacks.vot
from astropy.io.votable import parse_single_table

# read vot to pandas
table = parse_single_table('../data/all_stacks.vot')

# recover column names
df_all_stacks = table.to_table().to_pandas()
df_all_stacks.columns = [col.name for col in table.fields]

In [17]:
result_discrepant = analyze_match_cases(df_in_crit_test_and_more, df_all_stacks)

In [19]:
result_discrepant

{'N_yNWAY': 1006, 'N_OK': 952, 'N_NoML': 54, 'N_FLIP': 44, 'N_NONE': 362}